In [1]:
import numpy as np
import pandas as pd

from datetime import datetime as dt
import random
import re

from tqdm import tqdm

from itertools import chain
from collections import Counter

In [2]:
data = pd.read_pickle("../../Data/app_opp.pickle")
data.sample(4)

,ApplicationId,OpportunityId,Title,ExternalDescription,WhatYouDid,dateAppliedAt,JobTitle,LicenseAndCertification,skill,major,degree
48797,vIMeeUaRMkSMFXFG3tmWiA==,yWgqsBogkEOuTYIsvNdlJA==,"ACNO Laredo Medical Center - Laredo, TX",<p><em>Laredo Medical Center is Laredo&rsquo;s...,\nUHS South Texas Health System McAllen Medica...,2022-10-05 07:20:01.253,Director Emergency Department,NRP,Pharmacy,Nursing,Post Graduate Studies Cardiovascular
31917,V6zWhoVMx02P9HqZkb2Q1Q==,CHEsCAwyeEmSxpNZy4Qf0A==,Teller,<table>\n<tbody>\n<tr>\n<td>\n<p><strong>Telle...,"- Present, promote and sell products/services ...",2019-08-03 07:03:55.250,Sales representative,IGETC,2nd language,International Business,Bachelors
63978,TTegDaiHQEeJBdrIOFuzIQ==,vepl7eCMr0mzIomfOnUkZQ==,Development Coach,<p><strong>Department: Player Development</str...,• Responsible for on and off-site consulting s...,2021-11-10 10:49:10.430,Consultant,FACHE,Cooking,None,Certificate in Lean Healthcare Management Prin...
5938,vIS6V+bJqUCqqMZl9jHB3g==,sSrD6wXvKUSVhXKn3nHO3A==,Contract Administrator,<p><strong>Job Summary</strong></p>\n<p>BioLeg...,Assisted in drafting and negotiating of commer...,2019-05-29 01:32:28.292,Paralegal,Paralegal Certificate,Vendor agreements,Law,Certificate


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   ApplicationId            70000 non-null  object        
 1   OpportunityId            70000 non-null  object        
 2   Title                    70000 non-null  object        
 3   ExternalDescription      70000 non-null  object        
 4   WhatYouDid               65755 non-null  object        
 5   dateAppliedAt            70000 non-null  datetime64[ns]
 6   JobTitle                 70000 non-null  object        
 7   LicenseAndCertification  70000 non-null  object        
 8   skill                    70000 non-null  object        
 9   major                    43059 non-null  object        
 10  degree                   70000 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 5.9+ MB


---

<b>Fields related to opportunities:</b>
- OpportunityId
- Title
- ExternalDescirption

<b>Fields related to applications:</b>
- ApplicationId
- dateAppliedAt
- JobTitle
- WhatYouDid(free form text written by applicants and it serves as a summary of the work they did, they can fill it however they like)
- LicenseAndCertification
- major
- degree
- skill

In [4]:
data.nunique()

ApplicationId               175
OpportunityId               132
Title                       124
ExternalDescription         129
WhatYouDid                  661
dateAppliedAt               175
JobTitle                    612
LicenseAndCertification     247
skill                      1126
major                        68
degree                      195
dtype: int64

In [5]:
data.groupby('OpportunityId',as_index=False).nunique()['ApplicationId'].sum() # Confirming data structure

175

We're looking at 175 applications to 132 posts. A few postings have identical Titles to others. Even fewer have identical descriptions (reposts)

In [6]:
data.groupby('OpportunityId', as_index=False).nunique().sort_values(by='ApplicationId', ascending=False)

,OpportunityId,ApplicationId,Title,ExternalDescription,WhatYouDid,dateAppliedAt,JobTitle,LicenseAndCertification,skill,major,degree
78,dmlXNFI0MEOlW+qh07E4iQ==,17,1,1,86,17,88,30,141,16,39
123,wPH/PTSFV0SBmSzZlWnftA==,4,1,1,16,4,15,7,43,2,7
5,4xZy2W9hJkG3QJH7L/yLxw==,4,1,1,3,4,5,4,22,3,8
77,dFx5AVFlUUqQ5MW3Z0oQwg==,3,1,1,11,3,11,6,76,3,8
75,cn146CawdUKr/7Ha4/tMpA==,3,1,1,10,3,12,4,56,2,8
...,...,...,...,...,...,...,...,...,...,...,...
41,KURKW3m+Ik2y6BNjhBa2uA==,1,1,1,9,1,8,2,10,0,2
40,KHpHZBedt0m29dJZKnQd3Q==,1,1,1,1,1,3,1,2,1,1
39,JV9c2CbxEEiFZEXr07shmQ==,1,1,1,1,1,1,1,1,2,2
38,ItT0hWCka0GkeRDCHSOeEg==,1,1,1,7,1,5,2,26,0,1


The most applied to job received 17 applications.

In [7]:
data.groupby(['ApplicationId', 'dateAppliedAt'], as_index=False).nunique().sort_values(by='JobTitle', ascending=False).head()

,ApplicationId,dateAppliedAt,OpportunityId,Title,ExternalDescription,WhatYouDid,JobTitle,LicenseAndCertification,skill,major,degree
97,eymRGI00kEu533ZedmHMuA==,2020-08-28 19:26:46.875,1,1,1,11,17,1,17,2,2
100,gioI79njOEmfQX+HgFHGWQ==,2019-04-09 08:57:38.679,1,1,1,12,14,1,29,2,2
94,e/TH2fLIEUuNnbV4AkcAvA==,2023-01-03 22:57:21.829,1,1,1,10,10,2,50,0,4
25,9f1jcTY//UCBvE1JT+5u3g==,2020-09-07 20:34:39.781,1,1,1,9,10,1,5,3,4
99,gh2rm3WZY0mkjpsH+z9rsQ==,2020-09-20 21:37:49.219,1,1,1,10,10,3,4,2,3


Each application only went to one job (to be expected)

In [8]:
data.loc[data['ApplicationId'] == '9d0Ik6f0aUKmo6GF1lAqGw=='] #Sample application

,ApplicationId,OpportunityId,Title,ExternalDescription,WhatYouDid,dateAppliedAt,JobTitle,LicenseAndCertification,skill,major,degree
45434,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,IV push certification,Nursing,LPN Certificate
45435,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,IV insertion,Nursing,LPN Certificate
45436,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,Wound Care,Nursing,LPN Certificate
45437,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,Tube feeding,Nursing,LPN Certificate
45438,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,administering meds,Nursing,LPN Certificate
45439,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,administration of meds in GI tubes,Nursing,LPN Certificate
45440,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,Urinary Catheter insurtion,Nursing,LPN Certificate
45441,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,catheter care,Nursing,LPN Certificate
45442,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,trach care,Nursing,LPN Certificate
45443,9d0Ik6f0aUKmo6GF1lAqGw==,cOibKSS1H0GnB+tTy2Sc+w==,LPN - 4 South - FT - 7:00 PM-7:00 AM,<p>A Licensed Practical Nurse is responsible f...,I started off as a Nurse Intern for 10 months ...,2019-03-29 00:16:50.357,LPN,L-AL-MVR Drivers License,colostomy/ostomy care,Nursing,LPN Certificate


The application data a candidate submits is vast, so it is distributed across many rows. Individual skills, degrees, and experiences all have their own lines. This information can be aggregated to give an overall understanding of the candidate.

In [9]:
# Looking at randomly selected applicants' experience data

random_index = random.randint(0, len(data.WhatYouDid) - 1)
print(data.WhatYouDid[random_index])

Responsible for assisting the district curriculum and instruction department with the development,
implementation, monitoring, and evaluation of science curriculum for all high schools. Provided
campus support for administrators and teachers in generating reports and the disaggregation of student performance data for intervention and compliance with the district's accountability
performance goals. Performed other duties as assigned by the district including monitoring of state testing and intake of testing materials for verification.


### Notes: 
- ApplicationId and dateAppliedAt have the same number of unique values. that means every application was made at a different time. This is what we'd expect.
- There are multiple rows dedicated to each application as every skill, degree, certification, license and WhatYouDid gets its own row.
- Many applicants have held multiple roles (some of which held non-unique titles), others held no experience.
- Applicant info is duplicated heavily since the information is in row format. It will be aggregated on the application level.

In [10]:
data.loc[data.WhatYouDid.isna()].shape

(4245, 11)

In [11]:
data.groupby('ApplicationId', as_index=False).nunique().sort_values(by='OpportunityId', ascending=False)

,ApplicationId,OpportunityId,Title,ExternalDescription,WhatYouDid,dateAppliedAt,JobTitle,LicenseAndCertification,skill,major,degree
0,/eT9zghWr068l9Ba2k/TOg==,1,1,1,6,1,5,1,21,1,5
120,rY3Gn7dBwUOGUdZJmScz9Q==,1,1,1,7,1,5,1,17,3,5
112,m7wZYtTJN02bk3ARH/855Q==,1,1,1,3,1,3,1,14,1,1
113,m84E20F3j0Oh0PGgYUn9+w==,1,1,1,0,1,6,1,13,1,1
114,m8o0XcBAZU2mFt10Z7YfLA==,1,1,1,0,1,7,1,45,1,1
...,...,...,...,...,...,...,...,...,...,...,...
60,TSxV4DAUnUSu0qj0/LpbmQ==,1,1,1,4,1,4,1,12,1,2
61,TTEefLmJ4kKGMva97tJ8kA==,1,1,1,5,1,4,1,18,0,1
62,TTIor24kGUe3DjHpu+rSgA==,1,1,1,4,1,4,1,16,0,1
63,TTOloc6EBEGicB6K+6y6tQ==,1,1,1,1,1,1,1,1,0,1


In [12]:
data.groupby('OpportunityId', as_index=False).nunique().sort_values(by='ApplicationId', ascending=False)

,OpportunityId,ApplicationId,Title,ExternalDescription,WhatYouDid,dateAppliedAt,JobTitle,LicenseAndCertification,skill,major,degree
78,dmlXNFI0MEOlW+qh07E4iQ==,17,1,1,86,17,88,30,141,16,39
123,wPH/PTSFV0SBmSzZlWnftA==,4,1,1,16,4,15,7,43,2,7
5,4xZy2W9hJkG3QJH7L/yLxw==,4,1,1,3,4,5,4,22,3,8
77,dFx5AVFlUUqQ5MW3Z0oQwg==,3,1,1,11,3,11,6,76,3,8
75,cn146CawdUKr/7Ha4/tMpA==,3,1,1,10,3,12,4,56,2,8
...,...,...,...,...,...,...,...,...,...,...,...
41,KURKW3m+Ik2y6BNjhBa2uA==,1,1,1,9,1,8,2,10,0,2
40,KHpHZBedt0m29dJZKnQd3Q==,1,1,1,1,1,3,1,2,1,1
39,JV9c2CbxEEiFZEXr07shmQ==,1,1,1,1,1,1,1,1,2,2
38,ItT0hWCka0GkeRDCHSOeEg==,1,1,1,7,1,5,2,26,0,1


### Above we see that the important info here is the nunique(Application Id), as the rest are either info on the Opportunity or aggregated info across applicants

In [13]:
applicant_columns = ['ApplicationId', 'dateAppliedAt', 'JobTitle', "WhatYouDid", "LicenseAndCertification",
                     "major", "degree", "skill"]

opportunity_columns = ['OpportunityId', 'Title', 'ExternalDescription']

In [14]:
data.groupby(opportunity_columns).nunique().sample(2)

ApplicationId   
OpportunityId            Title             ExternalDescription                                                 
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...              1  \
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...              1   

                                                                                               WhatYouDid   
OpportunityId            Title             ExternalDescription                                              
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...           3  \
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...          10   

                                                                                               dateAppliedAt   
OpportunityId            Title             ExternalDescription                                                 
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...              1  \
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...              1   

                                                                                               JobTitle   
OpportunityId            Title             ExternalDescription                                            
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...         3  \
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...         9   

                                                                                               LicenseAndCertification   
OpportunityId            Title             ExternalDescription                                                           
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...                        1  \
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...                        1   

                                                                                               skill   
OpportunityId            Title             ExternalDescription                                         
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...      4  \
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...     38   

                                                                                               major   
OpportunityId            Title             ExternalDescription                                         
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...      0  \
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...      1   

                                                                                               degree  
OpportunityId            Title             ExternalDescription                                         
gOWTKiYjt0yTMPKvUM30lw== Store Associate   <p>Our store employees are the face of the ALDI...       2  
vepl7eCMr0mzIomfOnUkZQ== Development Coach <p><strong>Department: Player Development</stro...       5

In [18]:
# for each of these columns, make and print a dataframe of the value counts ['JobTitle	LicenseAndCertification	skill	major	degree]
data_dict = {}
for col in ['JobTitle', 'LicenseAndCertification', 'skill', 'major', 'degree']:
    data_dict[col] = pd.DataFrame(data[col].value_counts())
    print(data_dict[col])

                                           count
JobTitle                                        
Paralegal                                   2700
Student Nurse                               1584
Science Teacher/Coach                       1530
Registered Nurse                            1246
Science Teacher/Head Boys Basketball        1020
...                                          ...
Wrapper                                        1
Finance Counselor/ Admissions Rep              1
Pharmacy Technician/ Charge Reconciliator      1
Machine operator                               1
Deboner                                        1

[612 rows x 1 columns]
                                                    count
LicenseAndCertification                                  
Registered Nurse                                     2979
RN                                                   2292
FACHE                                                2090
Nursing Professional Development                 